### SpeechAnalyzer

## Progetto Technologies for Advanced Programming 2019/2020

- Aldo Fiorito X81000447
- Francesco Petrosino X81000533\
Studenti presso Università di Catania cdl Informatica

## Idea e scopo di SpeechAnalyzer
L'idea è nata osservando una community di persone all'interno dell'applicativo VOIP per le chat vocali Discord.\ La domanda principale posta è stata "Chi è il più chiacchierone del gruppo?"

Bisogna fare prima una premessa ...

![](train.jpg)

## Da qui SpeechAnalyzer

Fornisce statistiche e metriche su numerosi flussi vocali all'interno della propria rete

## Architettura dell'applicativo

### In breve
1. L'utente si registra all'applicativo
2. Un file json costruito dal suo flusso vocale viene mandato a un kafka-producer, avente come topic un campo   specificato dall'user
3. Spark , essendo anche kafka-consumer , processa ed elabora altre informazioni dal dato e costruisce un datafraframe attraverso sparksql 
4. La nuova struttura creata viene indicizzata su elasticsearch
5. Kibana si occupa di ottenere le metriche e le statistiche dai dati indicizzati attraverso un'interfaccia user-end

## Flusso operativo

L'utente,tramite una form, viene fatto registrare all'applicativo inserendo il proprio nome,la propria compagnia( che sarà il topic kafka) e la propria lingua d'appartenenza.\
**Perchè la lingua?**
Perchè l'analizzatore riesce ad elaborare qualiasi lingua ( al momento le 5 principali ).\
Ogni input verrà tradotto nella stessa lingua

### PreProcessing del dato
Un primo step di preprocessing dell'input vocale è quello di filtraggio del testo.\
Attraverso un file in locale di "badwords" verranno sostituite quelle parole considerate blasfeme o offensive
Successivamente , il testo filtrato viene tradotto in inglese
Il file json inviato al kafka-producer avrà questa struttura :\\ 
{"name":"saro" , "message" : "hello guys","topic" : "Tech","language" : "en","id":"75df85bf"}

Dove topic si riferisce al predict del Machine Learning